In [ ]:
class Value:
    def __init__(self):
        self.value = None
    @staticmethod
    def _prepare_value(value):
        return value * 10
    def __get__(self, obj, obj_type):
        return self.value
    def __set__(self, obj, value):
        print('set')
        self.value = self._prepare_value(value)

class Class:
    def __init__(self):
        self.attr = Value()

instance = Class()
instance.attr = 10
instance2 = Class()
instance2.attr = 20
print(instance.attr, instance2.attr)

10 20


In [ ]:
!python --version

Python 3.6.9


In [ ]:
class ImportantValue:
    def __init__(self):
        self.amount = None
    def __get__(self, obj, obj_type):
        return obj.__dict__['amount']
    def __set__(self, obj, value):
        print(value)
        with open('log.txt', 'a') as f:
            # print(value)
            f.write(str(value))
            
        obj.__dict__['amount'] = value

class Account:
    amount = ImportantValue()

    def __init__(self, amount):
        self.amount = amount

bobs_account = Account(200)
bobs_account2 = Account(400)

# with open('log.txt', 'r') as f:
#     print(f.read())

200
400


In [ ]:
print(bobs_account.amount, bobs_account2.amount)

200 400


In [ ]:
class User:
    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name
    @property
    def full_name(self):
        return f'{self.first_name} {self.last_name}'
amy = User('Amy', 'Jones')
print(amy.full_name)
print(User.full_name)

Amy Jones


```
Методы: __get__, __set__ и __delete__. Если хотя бы один из этих методов определён для объекта, то он становится дескриптором.
```

```
object.__get__(self, instance, owner)
self : Ссылка на экземпляр
instance : Экземпляр класса владельца дескриптора, либо None, если обращаются в контексте класса, а не экземпляра.
owner : Класс владельца дескриптора.
```

In [ ]:
class Property:
    def __init__(self, getter):
        print('self', self)
        print('getter', getter)
        self.getter = getter
    def __get__(self, obj, obj_type=None):
        print(type(self), type(obj), obj_type)
        if obj is None:
            return self
        return self.getter(obj)

In [ ]:
class Class:
    @property
    def original(self):
        return 'original'

    @Property
    def custom_sugar(self):
        return 'custom sugar'

    def custom_pure(self):
        return 'custom pure'

    custom_pure = Property(custom_pure)

self <__main__.Property object at 0x7f56b9de9f28>
getter <function Class.custom_sugar at 0x7f56b9e0ca60>
self <__main__.Property object at 0x7f56b9de96d8>
getter <function Class.custom_pure at 0x7f56b9d90bf8>


In [ ]:
obj = Class()
# print(obj.original)
print(obj.custom_sugar)
print(Class.custom_sugar)

<class '__main__.Property'> <class '__main__.Class'> <class '__main__.Class'>
custom sugar
<class '__main__.Property'> <class 'NoneType'> <class '__main__.Class'>


In [ ]:
class StaticMethod:
    def __init__(self, func):
        self.func = func
        
    def __get__(self, obj, obj_type=None):
        return self.func

class ClassMethod:
    def __init__(self, func):
        self.func = func

    def __get__(self, obj, obj_type=None):
        if obj_type is None:
            obj_type = type(obj)

        def new_func(*args, **kwargs):
            return self.func(obj_type, *args, **kwargs)

        return new_func

In [4]:
class NonNegative:
    def __init__(self, name):
        self.name = name  # (4)
    def __get__(self, instance, owner):
        print('instance._name', instance._name)
        print('owner', owner)
        print('instance.__dict__', instance.__dict__)
        return instance.__dict__[self.name]  # (5)
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Cannot be negative.')
        instance.__dict__[self.name] = value  # (6)

class Order:
    price = NonNegative('price')  # (3)
    quantity = NonNegative('quantity')
    def __init__(self, name, price, quantity):
        # print('init')
        self._name = name
        self.price = price
        self.quantity = quantity
    def total(self):
        return self.price * self.quantity
print('create')
apple_order = Order('apple', 1, 10)
print('total')
apple_order.total()
# 10
# apple_order.price = -10
# ValueError: Cannot be negative
# apple_order.quantity = -10
# ValueError: Cannot be negative

create
total
instance._name apple
owner <class '__main__.Order'>
instance.__dict__ {'_name': 'apple', 'price': 1, 'quantity': 10}
instance._name apple
owner <class '__main__.Order'>
instance.__dict__ {'_name': 'apple', 'price': 1, 'quantity': 10}


10

In [ ]:
apple_order2 = Order('apple', 2, 30)
apple_order2.total()

AttributeError: ignored

In [ ]:
apple_order.total()

10

In [ ]:
apple_order2.total()

60

In [ ]:
Order.price

AttributeError: ignored

In [26]:
def dec(func):
    print('dec')
    def decorator():
        print("1")
        func()
        print("2")
    return decorator

@dec
def deco():
    print('ffffffff')

dec


In [27]:
deco()

1
ffffffff
2


In [29]:
def dec():
    print('dec1')
    def decorator(func):
        print('dec2')
        def dec2():
            print("1")
            func()
            print("2")
        return dec2
    return decorator

@dec()
def deco():
    print('ffffffff')

dec1
dec2


In [30]:
deco()

1
ffffffff
2


In [32]:
def f1(a):
    print('1', a)
    return a+1

def f2(a):
    print('2', a)
    return a+1

def f3(a):
    print('3', a)
    return a

f = f3(f2(f1(5)))

1 5
2 6
3 7


In [33]:
class disp():
    def dec():
        print('dec')
        def decorator(func):
            print('decorator', func.__name__)
            def dec2():
                print("1")
                func()
                print("2")
            return dec2
        return decorator

@disp.dec()
def echo():
    print('3')

dec
decorator echo


In [34]:
echo()

1
3
2
